In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'/Users/lakshitjain/Developer/ML Projects/Brain-Tumor-MRI-Classification'

In [36]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir : Path
    trained_model_path:Path
    base_model_path:Path
    training_data:Path
    param_epochs : int
    param_batch_size : int
    param_is_augmentation : bool
    param_image_size : list
    param_learning_rate : float
    param_seed : int

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    checkpoint_model_filepath: Path
    tensorboard_root_log_dir: Path

In [37]:
from brainTumorMRIClassification.constants import *
from brainTumorMRIClassification.utils.common import read_yaml, create_directories
import tensorflow as tf
from brainTumorMRIClassification import logger

In [38]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config=self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir), Path(config.tensorboard_root_log_dir)])
        
        return PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir)
        )
    
    def get_training_config(self)->TrainingConfig:
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model
        params=self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir)
        create_directories([Path(training.root_dir)])

        return TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            base_model_path=Path(prepare_base_model.base_model_path),
            training_data=Path(training_data),
            param_epochs=params.EPOCHS,
            param_batch_size=params.BATCH_SIZE,
            param_is_augmentation=params.AUGMENTATION,
            param_image_size=params.IMAGE_SIZE,
            param_learning_rate=params.LEARNING_RATE,
            param_seed=params.SEED
        )



In [39]:

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    @property
    def _create_early_stopping(self):
        return tf.keras.callbacks.EarlyStopping(
            monitor='loss',
            min_delta=1e-9,
            patience=8,
            verbose=True
        )

    @property
    def _create_reduce_lr(self):
        return tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.3,
            patience=5,
            verbose=True
        )

    def get_all_callbacks(self):
        return [
            self._create_early_stopping,
            self._create_reduce_lr
        ]


In [ ]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.base_model_path
        )
        self.model.compile(
            optimizer = tf.keras.optimizers.Adam(learning_rate=self.config.param_learning_rate, beta_1=0.869, beta_2=0.995),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']
        )

    def train_valid_generator(self):
        train_dir = os.path.join(self.config.training_data, "Training")
        test_dir = os.path.join(self.config.training_data, "Testing")
        image_size = self.config.param_image_size
        target_size = tuple(image_size[:2])  # Only height and width for flow_from_directory
        batch_size = self.config.param_batch_size
        SEED = self.config.param_seed
        from tensorflow.keras.preprocessing.image import ImageDataGenerator
        # Data augmentation and preprocessing for training
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=10,
            brightness_range=(0.85, 1.15),
            width_shift_range=0.002,
            height_shift_range=0.002,
            shear_range=12.5,
            zoom_range=0,
            horizontal_flip=True,
            vertical_flip=False,
            fill_mode="nearest"
        )
        self.train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=target_size,
            batch_size=batch_size,
            class_mode="categorical",
            seed=SEED
        )

        # No augmentation for test data, just rescaling
        test_datagen = ImageDataGenerator(rescale=1./255)
        self.valid_generator = test_datagen.flow_from_directory(
            test_dir,
            target_size=target_size,
            batch_size=batch_size,
            class_mode="categorical",
            shuffle=False,
            seed=SEED
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        logger.info(f"Model saved at {path}")

    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        logger.info("Training Started")
        self.model.fit(
            self.train_generator,
            epochs=self.config.param_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [41]:
try:
    config=ConfigurationManager()
    prepare_callbacks_config=config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callbacks_list = prepare_callbacks.get_all_callbacks()

    training_config=config.get_training_config()
    training=Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list=callbacks_list)
except Exception as e:
    raise e


[2025-08-24 19:08:21,066] : INFO : common : YAML file config/config.yaml loaded successfully.
[2025-08-24 19:08:21,069] : INFO : common : YAML file params.yaml loaded successfully.
[2025-08-24 19:08:21,069] : INFO : common : Created directory: artifacts
[2025-08-24 19:08:21,070] : INFO : common : Created directory: artifacts/prepare_callbacks/checkpoint_dir
[2025-08-24 19:08:21,070] : INFO : common : Created directory: artifacts/prepare_callbacks/tensorboard_log_dir
[2025-08-24 19:08:21,071] : INFO : common : Created directory: artifacts/training
[2025-08-24 19:08:21,069] : INFO : common : YAML file params.yaml loaded successfully.
[2025-08-24 19:08:21,069] : INFO : common : Created directory: artifacts
[2025-08-24 19:08:21,070] : INFO : common : Created directory: artifacts/prepare_callbacks/checkpoint_dir
[2025-08-24 19:08:21,070] : INFO : common : Created directory: artifacts/prepare_callbacks/tensorboard_log_dir
[2025-08-24 19:08:21,071] : INFO : common : Created directory: artifac

/Users/lakshitjain/Developer/ML Projects/Brain-Tumor-MRI-Classification/venv/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8125 - loss: 0.2888 - val_accuracy: 0.7414 - val_loss: 0.7053 - learning_rate: 0.0010
Epoch 3/40
Epoch 3/40
178/178 ━━━━━━━━━━━━━━━━━━━━ 25s 141ms/step - accuracy: 0.7952 - loss: 0.5052 - val_accuracy: 0.7898 - val_loss: 0.5689 - learning_rate: 0.0010
Epoch 4/40
178/178 ━━━━━━━━━━━━━━━━━━━━ 25s 141ms/step - accuracy: 0.7952 - loss: 0.5052 - val_accuracy: 0.7898 - val_loss: 0.5689 - learning_rate: 0.0010
Epoch 4/40
178/178 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9062 - loss: 0.4078 - val_accuracy: 0.7859 - val_loss: 0.5835 - learning_rate: 0.0010
Epoch 5/40
178/178 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9062 - loss: 0.4078 - val_accuracy: 0.7859 - val_loss: 0.5835 - learning_rate: 0.0010
Epoch 5/40
178/178 ━━━━━━━━━━━━━━━━━━━━ 28s 155ms/step - accuracy: 0.8440 - loss: 0.3884 - val_accuracy: 0.7961 - val_loss: 0.5455 - learning_rate: 0.0010
Epoch 6/40
178/178 ━━━━━━━━━━━━━━━━━━━━ 28s 155ms/step - accuracy: 0.8440 - l

2025-08-24 19:14:56.291838: W tensorflow/core/framework/op_kernel.cc:1842] UNKNOWN: FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/data_ingestion/brain-mri/Training/pituitary/Tr-pi_1397.jpg'
Traceback (most recent call last):

  File "/Users/lakshitjain/Developer/ML Projects/Brain-Tumor-MRI-Classification/venv/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)

  File "/Users/lakshitjain/Developer/ML Projects/Brain-Tumor-MRI-Classification/venv/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/lakshitjain/Developer/ML Projects/Brain-Tumor-MRI-Classification/venv/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/lakshitjain/Developer/ML Projects/Brain-Tumor-MRI-Classification/venv/lib

 34/178 ━━━━━━━━━━━━━━━━━━━━ 28s 197ms/step - accuracy: 0.9772 - loss: 0.0699

2025-08-24 19:14:56.883519: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: UNKNOWN: FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/data_ingestion/brain-mri/Training/pituitary/Tr-pi_1397.jpg'
Traceback (most recent call last):

  File "/Users/lakshitjain/Developer/ML Projects/Brain-Tumor-MRI-Classification/venv/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)

  File "/Users/lakshitjain/Developer/ML Projects/Brain-Tumor-MRI-Classification/venv/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/lakshitjain/Developer/ML Projects/Brain-Tumor-MRI-Classification/venv/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/lakshitjain/Developer/ML 

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/data_ingestion/brain-mri/Training/pituitary/Tr-pi_1397.jpg'
Traceback (most recent call last):

  File "/Users/lakshitjain/Developer/ML Projects/Brain-Tumor-MRI-Classification/venv/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)

  File "/Users/lakshitjain/Developer/ML Projects/Brain-Tumor-MRI-Classification/venv/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/lakshitjain/Developer/ML Projects/Brain-Tumor-MRI-Classification/venv/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/lakshitjain/Developer/ML Project